In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%config Completer.use_jedi = False  # 자동완성 속도 증가
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
from keras import layers

Using TensorFlow backend.


In [4]:
# 보스턴 주택 데이터셋 로드
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = (
    boston_housing.load_data())

In [5]:
# 데이터 정규화 (테스트 데이터는 정규화에 사용하지 말 것)
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [8]:
# 모델 정의 (스칼라 회귀를 위한 구성)
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1) # 활성화 함수 X
    ])
    model.compile(optimizer='rmsprop',
                 loss='mse',
                 metrics=['mae'])
    return model

In [10]:
# k겹 검증
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print(f'#{i + 1}번째 폴드 처리중')
    val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples]

#1번째 폴드 처리중
#2번째 폴드 처리중
#3번째 폴드 처리중
#4번째 폴드 처리중
